In [3]:
import jax
import jax.numpy as jnp
from jaxlie import SE3, SO3

from sdf_world.sdf_world import *
from sdf_world.robots import *

from network import *
from loss import *

In [4]:
world = SDFWorld()


You can open the visualizer by visiting the following URL:
http://127.0.0.1:7004/static/


In [5]:
world.show_in_jupyter()

In [6]:
HAND_URDF = "./sdf_world/assets/robots/panda/hand.urdf"
hand_model = RobotModel(HAND_URDF, PANDA_PACKAGE, is_floating=True)
panda_model = RobotModel(PANDA_URDF, PANDA_PACKAGE)

In [7]:
frame = Frame(world.vis, "frame", 0.2)
hand = Robot(world.vis, "hand", hand_model)
shoulder = Sphere(world.vis, "shoulder", 0.1, "red", 0.5)
elbow = Sphere(world.vis, "elbow", 0.1, "red", 0.5)
wrist = Sphere(world.vis, "wrist", 0.1, "red", 0.5)
shoulder.set_translate([0,0,0.33])
elbow.set_translate([0,0,0.66])
hand.set_translate([0,0,1])

In [10]:
# problem
qrand = np.random.uniform(panda_model.lb, panda_model.ub)
pose_rand = SE3(panda_model.fk_fn(qrand)[-1])

#vis
frame.set_pose(pose_rand)
hand_base = pose_rand @ SE3.from_translation(jnp.array([0,0,-0.105]))
hand.set_pose(hand_base)

In [22]:
from network import *
feas_fn = get_mlp_by_path("elbow_feas_net1.pth")

ee_pose = SE3(panda_model.fk_fn(panda_model.neutral)[-1])
wrist_pos = panda_model.fk_fn(panda_model.neutral)[7][-3:]
wrist_wrt_ee = ee_pose.inverse().apply(wrist_pos)

In [32]:
ws_lb = [-0.8, -0.8, -0.3]
ws_ub = [0.8, 0.8, 1.3]
p_rand = np.random.uniform(ws_lb, ws_ub, size=(100, 3))

In [33]:
def elbow_feasiblility(ee_pose:SE3, elbow):
    wrist = ee_pose.apply(wrist_wrt_ee)
    x = jnp.hstack([wrist, elbow])
    return feas_fn(x)
elbow_feasiblility_batch = jax.vmap(elbow_feasiblility, in_axes=(None,0))

In [39]:
logits = elbow_feasiblility_batch(pose_rand, p_rand)
feasibility = nn.sigmoid(logits.max())
elbow_init = p_rand[logits.argmax()]
print(f"feasibility:{feasibility}")

feasibility:0.9999997615814209


In [38]:
p_wrist = pose_rand.apply(wrist_wrt_ee)
wrist.set_translate(p_wrist)

In [43]:
elbow.set_translate(p_rand[2])

In [15]:
hand_model.links["panda_hand"].has_mesh

True

In [ ]:
feas_fn = get_mlp_by_path("elbow_feas_net1.pth")